In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
import json 
base_dir = 'dataset\\LBNL_FDD_Dataset_SDAHU\\'
import pickle
from sklearn.metrics import precision_recall_curve, auc,confusion_matrix
import json
from sklearn.model_selection import TimeSeriesSplit
import random
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.stattools import adfuller
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor

In [22]:
def load_data(file_name):
    df = pd.read_csv(base_dir+file_name, index_col='Datetime')
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
    return df

def plot_line_graph(df1, df2, feature):
    plt.plot(df1[feature].values, 'g')
    plt.plot(df2[feature].values, 'r')
    plt.show()


def check_stationarity(data, column_name, significance_level=0.05):
    """
    Check the stationarity of a time series using the Augmented Dickey-Fuller (ADF) test.

    Parameters:
    - data: pd.DataFrame
        Input DataFrame with time series data.
    - column_name: str
        Name of the column containing the time series data.
    - significance_level: float, optional (default=0.05)
        Significance level for the ADF test.

    Returns:
    - bool
        True if the time series is stationary, False otherwise.
    """

    # Check if the specified column exists in the DataFrame
    if column_name not in data.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame.")

    try:
        # Perform the ADF test
        result = adfuller(data[column_name])

        # Extract ADF test statistic and p-value
        adf_statistic = result[0]
        p_value = result[1]
    
        # Compare p-value with significance level
        is_stationary = p_value <= significance_level
        
        if stationary:
            print(f"The time series is stationary. ADF Statistic: {adf_statistic}, p-value: {p_value}")
        else:
            print(f"The time series is not stationary. ADF Statistic: {adf_statistic}, p-value: {p_value}")
            
        #return is_stationary, adf_statistic, p_value
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def drop_low_variance_features(data, threshold=0.01):
    
    # Calculate variance of each feature
    variances = data.var()

    #print("Variances of features:")
    #print(variances)

    # Use VarianceThreshold to identify low variance features
    selector = VarianceThreshold(threshold)
    selector.fit(data)

    # Get indices of features to keep
    keep_indices = selector.get_support(indices=True)

    # Subset the DataFrame with selected features
    selected_data = data.iloc[:, keep_indices]

    return selected_data


def evaluate_classification_metrics(y_true, y_pred):
    """
    Evaluate classification metrics such as accuracy, precision, recall, and F1 score.

    Parameters:
    - y_test: list or array-like
        True labels.
    - y_pred: list or array-like
        Predicted labels.

    Returns:
    - dict
        Dictionary containing evaluation metrics.
    """
    # Initialize counts
    TP = TN = FP = FN = 0

    # Calculate confusion matrix elements
    for true, predicted in zip(y_true, y_pred):
        if true == 1 and predicted == 1:
            TP += 1
        elif true == 0 and predicted == 0:
            TN += 1
        elif true == 0 and predicted == 1:
            FP += 1
        elif true == 1 and predicted == 0:
            FN += 1

    print("TP:", TP)
    print("TN:", TN)
    print("FP:", FP)
    print("FN:", FN)

    # Compute classification metrics
    accuracy = round(accuracy_score(y_true, y_pred), 2)
    precision = round(precision_score(y_true, y_pred))
    recall = round(recall_score(y_true, y_pred), 2)
    f1 = round(f1_score(y_true, y_pred), 2)

    # Create a dictionary to store the metrics
    metrics_dict = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    return metrics_dict

def fine_tune_model(param_grid, model, X):
    # Create GridSearchCV object
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=True)

    # Perform grid search on non-faulty data
    grid_search.fit(X)

    # Get the best Isolation Forest model
    best_model = grid_search.best_estimator_
    
    return best_model

def cv_for_test_set_eval(dict_fault, X, freq, splits, model_name):
    tscv = TimeSeriesSplit(n_splits=splits)  # You can adjust the number of splits
    results = {}
    counter = 1
    for file in dict_fault.keys():
        if 'short' in file:
            continue
        else:
            results[file] = {}
            results[file]['Accuracy'] = []
            results[file]['F1'] = []
            results[file]['Precision'] = []
            results[file]['Recall'] = []
            
    for train_index, test_index in tscv.split(X):
        
        X_train = X[train_index]
        train_fft = np.fft.fft(X_train)
        scaler = MinMaxScaler()
        train_scaled_fft = scaler.fit_transform(train_fft.real)

        # Train the specified binary classifier
        if model_name == 'lof':
            param_grid = {'n_neighbors': [5, 10, 15, 20, 25, 30, 35, 40], 'contamination': [0.01, 0.05, 0.1, 0.2]}
            lof = LocalOutlierFactor(novelty=True)
            trained_model = fine_tune_model(param_grid, lof, train_scaled_fft)
            
        elif model_name == 'isolation_forest':
            param_grid = {
                        'n_estimators': [50, 100, 150, 200, 250, 300],
                        'max_samples': ['auto', 0.5, 0.7],
                        'contamination': [0.05, 0.1, 0.2],
                        'max_features': [1.0, 0.8, 0.6, 0.4, 0.3, 0.2],
                        'bootstrap': [True, False]
                        }
            # Initialize Isolation Forest
            isolation_forest = IsolationForest(random_state=42)
            trained_model = fine_tune_model(param_grid, isolation_forest, train_scaled_fft)

        
        elif model_name == 'elliptic':
            param_grid = {
                        'contamination': [0.05, 0.1, 0.2],
                        'store_precision': [True, False],
                        'assume_centered': [True, False],
                        'support_fraction': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
                        }

            # Initialize Elliptic Envelope
            elliptic_envelope = EllipticEnvelope()
            trained_model = fine_tune_model(param_grid, elliptic_envelope, train_scaled_fft)
        
        elif model_name == 'onesvm':
            # Initialize One-Class SVM        
            param_grid = {
                'nu': [0.01, 0.05, 0.1, 0.2],
                'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
                'gamma': ['scale', 'auto']
                
            }
            one_class_svm = OneClassSVM()
            trained_model = fine_tune_model(param_grid, one_class_svm, train_scaled_fft)
        
        else:
            raise ValueError("Invalid model_name. Choose from 'log', or 'isolation_forest'.")
        
        print(trained_model)
        for file in dict_fault.keys():
            if 'short' in file:
                continue
            else:
                df_test_resampled = dict_fault[file].resample(freq).mean()
               
                
                X_test = df_test_resampled.values[test_index]
                X_non_faulty = X[test_index]
                
                X_test_fault_non_fault = np.vstack((X_test, X_non_faulty))
                
                y_test = [1]*X_test.shape[0]
                y_test.extend([0]*X_non_faulty.shape[0])
                
                test_fft = np.fft.fft(X_test_fault_non_fault)
                test_pred_labels = trained_model.predict(scaler.transform(test_fft.real))
                predicted_labels = np.where(test_pred_labels == -1, 1, 0)
                
                metrics = evaluate_classification_metrics(y_test, predicted_labels)
                print(counter, metrics)
                results[file]['Accuracy'].append(metrics['Accuracy'])
                results[file]['F1'].append(metrics['F1 Score'])
                results[file]['Precision'].append(metrics['Precision'])
                results[file]['Recall'].append(metrics['Recall'])
        print('*'*100)
        counter+=1
    return results
    

In [4]:
correct_data = load_data('AHU_annual.csv')

In [5]:
#df_correct_dropped_variance_features = drop_low_variance_features(correct_data)

In [6]:
#selected_features = list(df_correct_dropped_variance_features.keys())

In [7]:
dict_faulty_df = {}
for c, file in enumerate(os.listdir(base_dir)):
    if c>0:
        dict_faulty_df[file] = load_data(file)
        #dict_faulty_df[file] = df[selected_features]

In [8]:
training_start_date_time = '2018-01-01 01:00:00'
training_end_date_time = '2018-10-31 23:59:00'

test_start_data = '2018-11-01 00:00:00'

In [9]:
resampling_freq = '5T'

In [10]:
df_train_resampled = correct_data.resample(resampling_freq).mean()
X = df_train_resampled.values

### 10 Fold Time Series Evaluation of OCC Models

#### Local Outlier Factor (LOF)

In [ ]:
test_set_cv_results_lof = cv_for_test_set_eval(dict_faulty_df, X, resampling_freq, 10, 'lof')

#### Isolation Forest

In [ ]:
test_set_cv_results_isolation_forest = cv_for_test_set_eval(dict_faulty_df, X, resampling_freq, 10, 'isolation_forest')

#### Elliptic Envelope

In [ ]:
test_set_cv_results_eliptic = cv_for_test_set_eval(dict_faulty_df, X, resampling_freq, 10, 'elliptic')

#### One Class SVM

In [ ]:
test_set_cv_results_onesvm = cv_for_test_set_eval(dict_faulty_df, X, resampling_freq, 10, 'onesvm')

### Save 10 Fold Evaluation Results

In [ ]:
with open('results/10_fold_faults_non_faults_evaluation_lof.json', "w") as outfile: 
        json.dump(test_set_cv_results_lof, outfile, indent=4)
with open('results/10_fold_faults_non_faults_evaluation_isolation.json', "w") as outfile: 
        json.dump(test_set_cv_results_isolation_forest, outfile, indent=4)
with open('results/10_fold_faults_non_faults_evaluation_eliptic.json', "w") as outfile: 
        json.dump(test_set_cv_results_eliptic, outfile, indent=4)
with open('results/10_fold_faults_non_faults_evaluation_onesvm.json', "w") as outfile: 
        json.dump(test_set_cv_results_onesvm, outfile, indent=4)

### Trained Best Model
In our case, Eliptic Envelope performs, on average, better than the other OCC models.

In [45]:
df_train = correct_data[training_start_date_time : training_end_date_time].resample(resampling_freq).mean()
X_train = df_train.values

model_eliptic_envelop = EllipticEnvelope(contamination=0.05)
X_train_fft = np.fft.fft(X_train)

scaler = MinMaxScaler()
X_train_fft_scaled = scaler.fit_transform(X_train_fft.real)

model_eliptic_envelop.fit(X_train_fft_scaled)

EllipticEnvelope(contamination=0.05)

In [52]:
def train_evaluate_model(df_faulty_dict, model, scaler):
    
    for file in df_faulty_dict.keys():
        print(file)
        df_test = df_faulty_dict[file][test_start_data : ].resample(resampling_freq).mean()

        X_test = df_test.values
        X_test_fft = np.fft.fft(X_test)
    
        X_test_fft_scaled = scaler.transform(X_test_fft.real)
        
        y_test = np.asarray([1]*X_test.shape[0])
    
        test_pred_labels = model.predict(scaler.transform(X_test_fft_scaled.real))
        predicted_labels = np.where(test_pred_labels == -1, 1, 0)
        
        metrics = evaluate_classification_metrics(y_test, predicted_labels)
        print(metrics)

In [53]:
train_evaluate_model(dict_faulty_df, model_eliptic_envelop, scaler)

coi_bias_-2_annual.csv
TP: 17568
TN: 0
FP: 0
FN: 0
{'Accuracy': 1.0, 'Precision': 1, 'Recall': 1.0, 'F1 Score': 1.0}
coi_bias_-4_annual.csv
TP: 17568
TN: 0
FP: 0
FN: 0
{'Accuracy': 1.0, 'Precision': 1, 'Recall': 1.0, 'F1 Score': 1.0}
coi_bias_2_annual.csv
TP: 17568
TN: 0
FP: 0
FN: 0
{'Accuracy': 1.0, 'Precision': 1, 'Recall': 1.0, 'F1 Score': 1.0}
coi_bias_4_annual.csv
TP: 17568
TN: 0
FP: 0
FN: 0
{'Accuracy': 1.0, 'Precision': 1, 'Recall': 1.0, 'F1 Score': 1.0}
coi_leakage_010_annual.csv
TP: 17568
TN: 0
FP: 0
FN: 0
{'Accuracy': 1.0, 'Precision': 1, 'Recall': 1.0, 'F1 Score': 1.0}
coi_leakage_025_annual.csv
TP: 17568
TN: 0
FP: 0
FN: 0
{'Accuracy': 1.0, 'Precision': 1, 'Recall': 1.0, 'F1 Score': 1.0}
coi_leakage_040_annual.csv
TP: 17568
TN: 0
FP: 0
FN: 0
{'Accuracy': 1.0, 'Precision': 1, 'Recall': 1.0, 'F1 Score': 1.0}
coi_leakage_050_annual.csv
TP: 17568
TN: 0
FP: 0
FN: 0
{'Accuracy': 1.0, 'Precision': 1, 'Recall': 1.0, 'F1 Score': 1.0}
coi_stuck_010_annual.csv
TP: 17568
TN: 0
FP: 0
FN:

In [54]:
import joblib

# Save the model to a file
joblib.dump(model_eliptic_envelop, 'models/elliptic_envelope_model_'+str(resampling_freq)+'.pkl')
joblib.dump(scaler, 'models/minmax_elliptic_envelope_mode_'+str(resampling_freq)+'.pkl')

['models/minmax_elliptic_envelope_mode_5T.pkl']